In [1]:
import numpy as np
import pandas as pd
import itertools as it
import pickle
from scipy.stats import ttest_ind

# Read results

In [2]:
methods = ['mcmc_flat', 'mcmc_inter', 'mcmc_intra', 'msle_intra']
methods_pretty = {
    'mcmc_flat': 'MNL (MCMC)',
    'mcmc_inter': 'MXL-inter (MCMC)',
    'mcmc_intra': 'MXL-inter-intra (MCMC)',
    'msle_intra': 'MXL-inter-intra (MSL)'
}
R = 20
S_list = [1, 2]
N_list = [1000]
T_list = [10, 20]

metrics = [
    'time',
    'rmse_zetaMu', 'rmse_SigmaB', 'rmse_SigmaW',
    'probB_chosen_plugin',
    'probW_chosen_plugin',
    'brierB_plugin', 'brierW_plugin'
    ]

n_p = len(methods) * R * len(S_list) * len(N_list) * len(T_list)
df_results = pd.DataFrame(index=np.arange(n_p),
                          columns=[
                              'S', 'N', 'T', 'r', 'method',
                              *metrics
                              ])

i = -1
for m, S, N, T, r in it.product(methods, S_list, N_list, T_list, np.arange(R)):
    i += 1
    
    #Load results
    filename = 'results_{}_sim_S{}_N{}_T{}_r{}'.format(m,S,N,T,r)
    try:
        infile = open(filename, 'rb')
        res_tmp = pickle.load(infile)[0]
        infile.close()
    except:
        print(filename, " does not exist.")
    
    #Store results
    df_results['S'].iloc[i] = S
    df_results['N'].iloc[i] = N
    df_results['T'].iloc[i] = T
    df_results['r'].iloc[i] = r
    df_results['method'].iloc[i] = m
    
    df_results['time'].iloc[i] = res_tmp['time']
    
    if m not in ['mcmc_flat', 'mcmc_inter']:
        df_results['rmse_zetaMu'].iloc[i] = res_tmp['rmse_zetaMu']
        df_results['rmse_SigmaB'].iloc[i] = res_tmp['rmse_SigmaB']
        df_results['rmse_SigmaW'].iloc[i] = res_tmp['rmse_SigmaW']
    
    if m != 'msle_intra':
        
        df_results['probB_chosen_plugin'].iloc[i] = res_tmp['probB_chosen_plugin']
        df_results['probW_chosen_plugin'].iloc[i] = res_tmp['probW_chosen_plugin']
        
        df_results['brierB_plugin'].iloc[i] = res_tmp['brierB_plugin']
        df_results['brierW_plugin'].iloc[i] = res_tmp['brierW_plugin']

# Aggregate results

In [3]:
df_results[metrics] = df_results[metrics].astype('float')

stats = ['Mean', 'SE [\%]']
stats_func = {
    'Mean': lambda x: x.mean(),
    'SE [\%]': lambda x: x.std() / np.sqrt(x.size) if x.name == 'time' \
        else x.std() / np.sqrt(x.size) * 100
}

df_results_agg = {}
for s in stats:
    df_results_agg[s] = df_results.groupby(['S', 'N', 'T', 'method'])[metrics].agg(stats_func[s]).reset_index()

df_results_agg['Mean']

,S,N,T,method,time,rmse_zetaMu,rmse_SigmaB,rmse_SigmaW,probB_chosen_plugin,probW_chosen_plugin,brierB_plugin,brierW_plugin
0,1,1000,10,mcmc_flat,83.287675,NaN,NaN,NaN,0.393692,0.395085,0.201515,0.201297
1,1,1000,10,mcmc_inter,284.892175,NaN,NaN,NaN,0.401160,0.541033,0.198786,0.164565
2,1,1000,10,mcmc_intra,3422.865657,0.037442,0.072110,0.071369,0.403799,0.539405,0.198473,0.162690
3,1,1000,10,msle_intra,2932.694726,0.033627,0.061904,0.062708,NaN,NaN,NaN,NaN
4,1,1000,20,mcmc_flat,159.901484,NaN,NaN,NaN,0.397849,0.397225,0.199102,0.199983
5,1,1000,20,mcmc_inter,423.740944,NaN,NaN,NaN,0.405972,0.566568,0.196203,0.152037
6,1,1000,20,mcmc_intra,6135.585246,0.023194,0.036461,0.039033,0.409101,0.569576,0.195807,0.148899
7,1,1000,20,msle_intra,5003.757604,0.032437,0.043924,0.037850,NaN,NaN,NaN,NaN
8,2,1000,10,mcmc_flat,84.634956,NaN,NaN,NaN,0.393735,0.394997,0.201719,0.201266
9,2,1000,10,mcmc_inter,287.423598,NaN,NaN,NaN,0.401348,0.536439,0.198975,0.166909


# Format results
## Estimation accuracy

In [4]:
metrics = ['rmse_zetaMu', 'rmse_SigmaB', 'rmse_SigmaW']
methods = ['mcmc_intra', 'msle_intra']

df_results_agg_mask = {k: v[v['method'].isin(methods)].copy().reset_index() \
                       for k, v in df_results_agg.items()}

cols0 = [(i,i) for i in ['S', 'N', 'T', 'method']]
cols1 = [(i,j) for i in metrics for j in stats]
cols = [*cols0, *cols1]
df_results_format = pd.DataFrame(columns=pd.MultiIndex.from_tuples(cols))
for c in cols[:4]:
    df_results_format[c] = df_results_agg_mask[stats[0]][c[0]]
for m, s in it.product(metrics, stats):
    df_results_format[(m,s)] = df_results_agg_mask[s][m]
df_results_format[('method', 'method')].replace(methods_pretty, inplace=True)
df_results_format

S     N   T                  method rmse_zetaMu           rmse_SigmaB  \
   S     N   T                  method        Mean   SE [\%]        Mean   
0  1  1000  10  MXL-inter-intra (MCMC)    0.037442  0.491022    0.072110   
1  1  1000  10   MXL-inter-intra (MSL)    0.033627  0.310200    0.061904   
2  1  1000  20  MXL-inter-intra (MCMC)    0.023194  0.268236    0.036461   
3  1  1000  20   MXL-inter-intra (MSL)    0.032437  0.321625    0.043924   
4  2  1000  10  MXL-inter-intra (MCMC)    0.043554  0.529864    0.085846   
5  2  1000  10   MXL-inter-intra (MSL)    0.040296  0.408876    0.073039   
6  2  1000  20  MXL-inter-intra (MCMC)    0.021187  0.288977    0.041489   
7  2  1000  20   MXL-inter-intra (MSL)    0.033683  0.370209    0.048785   

            rmse_SigmaW            
    SE [\%]        Mean   SE [\%]  
0  1.031914    0.071369  0.694382  
1  0.590302    0.062708  0.388017  
2  0.293240    0.039033  0.255451  
3  0.236257    0.037850  0.258837  
4  1.253348    0.078982  0.763317  
5  0.861819    0.073110  0.592302  
6  0.510601    0.039651  0.312864  
7  0.347420    0.040698  0.322803

In [5]:
df_results_format.drop(columns=df_results_format.columns[1], inplace=True)
latex = df_results_format.to_latex(escape=False, float_format="%.3f", index=False)
text_file = open("table_est.tex", "w")
text_file.write(latex)
text_file.close()

## Estimation time

In [6]:
metrics = ['time']
methods = ['mcmc_flat', 'mcmc_inter', 'mcmc_intra', 'msle_intra']

df_results_agg_mask = {k: v[v['method'].isin(methods)].copy().reset_index() \
                       for k, v in df_results_agg.items()}

cols0 = [(i,i) for i in ['S', 'N', 'T', 'method']]
cols1 = [(i,j) for i in metrics for j in stats]
cols = [*cols0, *cols1]
df_results_format = pd.DataFrame(columns=pd.MultiIndex.from_tuples(cols))
for c in cols[:4]:
    df_results_format[c] = df_results_agg_mask[stats[0]][c[0]]
for m, s in it.product(metrics, stats):
    df_results_format[(m,s)] = df_results_agg_mask[s][m]
df_results_format[('method', 'method')].replace(methods_pretty, inplace=True)
df_results_format  

S     N   T                  method         time            
    S     N   T                  method         Mean     SE [\%]
0   1  1000  10              MNL (MCMC)    83.287675    1.316595
1   1  1000  10        MXL-inter (MCMC)   284.892175    2.742049
2   1  1000  10  MXL-inter-intra (MCMC)  3422.865657   28.208026
3   1  1000  10   MXL-inter-intra (MSL)  2932.694726   86.910645
4   1  1000  20              MNL (MCMC)   159.901484    2.061007
5   1  1000  20        MXL-inter (MCMC)   423.740944    0.731290
6   1  1000  20  MXL-inter-intra (MCMC)  6135.585246   75.555246
7   1  1000  20   MXL-inter-intra (MSL)  5003.757604  122.231735
8   2  1000  10              MNL (MCMC)    84.634956    1.688711
9   2  1000  10        MXL-inter (MCMC)   287.423598    2.138426
10  2  1000  10  MXL-inter-intra (MCMC)  3792.727721   57.187331
11  2  1000  10   MXL-inter-intra (MSL)  3085.604379  162.717860
12  2  1000  20              MNL (MCMC)   186.155402    3.311606
13  2  1000  20        MXL-inter (MCMC)   413.694588    4.364596
14  2  1000  20  MXL-inter-intra (MCMC)  6706.439169   61.857219
15  2  1000  20   MXL-inter-intra (MSL)  4372.619227  155.981699

In [7]:
df_results_format.drop(columns=df_results_format.columns[1], inplace=True)
latex = df_results_format.to_latex(escape=False, float_format="%.1f", index=False)
text_file = open("table_time.tex", "w")
text_file.write(latex)
text_file.close()

## Predictive accuracy

In [8]:
metrics = [
    'brierB_plugin', 'brierW_plugin',
    'probB_chosen_plugin', 'probW_chosen_plugin'
    ]
methods = ['mcmc_flat', 'mcmc_inter', 'mcmc_intra']

df_results_agg_mask = {k: v[v['method'].isin(methods)].copy().reset_index() \
                       for k, v in df_results_agg.items()}

cols0 = [(i,i) for i in ['S', 'N', 'T', 'method']]
cols1 = [(i,j) for i in metrics for j in stats]
cols = [*cols0, *cols1]
df_results_format = pd.DataFrame(columns=pd.MultiIndex.from_tuples(cols))
for c in cols[:4]:
    df_results_format[c] = df_results_agg_mask[stats[0]][c[0]]
for m, s in it.product(metrics, stats):
    df_results_format[(m,s)] = df_results_agg_mask[s][m]
df_results_format[('method', 'method')].replace(methods_pretty, inplace=True)
df_results_format

S     N   T                  method brierB_plugin           brierW_plugin  \
    S     N   T                  method          Mean   SE [\%]          Mean   
0   1  1000  10              MNL (MCMC)      0.201515  0.200447      0.201297   
1   1  1000  10        MXL-inter (MCMC)      0.198786  0.222329      0.164565   
2   1  1000  10  MXL-inter-intra (MCMC)      0.198473  0.234642      0.162690   
3   1  1000  20              MNL (MCMC)      0.199102  0.155669      0.199983   
4   1  1000  20        MXL-inter (MCMC)      0.196203  0.170263      0.152037   
5   1  1000  20  MXL-inter-intra (MCMC)      0.195807  0.181036      0.148899   
6   2  1000  10              MNL (MCMC)      0.201719  0.206285      0.201266   
7   2  1000  10        MXL-inter (MCMC)      0.198975  0.219696      0.166909   
8   2  1000  10  MXL-inter-intra (MCMC)      0.198659  0.224318      0.163968   
9   2  1000  20              MNL (MCMC)      0.201942  0.197422      0.205828   
10  2  1000  20        MXL-inter (MCMC)      0.199604  0.201472      0.151877   
11  2  1000  20  MXL-inter-intra (MCMC)      0.199310  0.208729      0.150187   

             probB_chosen_plugin           probW_chosen_plugin            
     SE [\%]                Mean   SE [\%]                Mean   SE [\%]  
0   0.228838            0.393692  0.310942            0.395085  0.371874  
1   0.318337            0.401160  0.336197            0.541033  0.606354  
2   0.306271            0.403799  0.361374            0.539405  0.580875  
3   0.186682            0.397849  0.246166            0.397225  0.277645  
4   0.392365            0.405972  0.274089            0.566568  0.708208  
5   0.391529            0.409101  0.296266            0.569576  0.707547  
6   0.225984            0.393735  0.339467            0.394997  0.374502  
7   0.293124            0.401348  0.350904            0.536439  0.541838  
8   0.285255            0.404072  0.355472            0.535969  0.511719  
9   0.208454            0.394133  0.329284            0.387921  0.346899  
10  0.391321            0.401061  0.320665            0.565419  0.662158  
11  0.379950            0.403841  0.327265            0.565685  0.635634

In [9]:
df_results_format.drop(columns=df_results_format.columns[1], inplace=True)
latex = df_results_format.to_latex(escape=False, float_format="%.3f", index=False)
text_file = open("table_pred.tex", "w")
text_file.write(latex)
text_file.close()

# Test results

In [10]:
def t_test(S, T, m1, m2, v1, v2):
    exp_mask = (df_results['S'] == S) & (df_results['T'] == T)
    cat1 = df_results[exp_mask & (df_results['method'] == m1)]
    cat2 = df_results[exp_mask & (df_results['method'] == m2)]
    print(ttest_ind(cat1[v1], cat2[v2]))
    
t_test(2, 10, 'mcmc_inter', 'mcmc_flat', 'brierW_plugin', 'brierW_plugin')

Ttest_indResult(statistic=-9.282591504341873, pvalue=2.592140825783968e-11)
